# 使用VLMEvalKit进行多模态模型评估

VLMEvalKit (python 包名为 vlmeval) 是一款专为大型视觉语言模型 (Large Vision-Language Models， LVLMs) 评测而设计的开源工具包。该工具支持在各种基准测试上对大型视觉语言模型进行一键评估，无需进行繁重的数据准备工作，让评估过程更加简便。

以下展示两种方式进行模型评估：
1. 使用EvalScope封装的VLMEvalKit评测接口
2. 直接使用VLMEvalKit评测接口

## 1. 使用EvalScope封装的VLMEvalKit评测接口

[EvalScope](https://github.com/modelscope/evalscope) 是魔搭社区官方推出的模型评估与性能基准测试框架，内置多个常用测试基准和评估指标，如MMLU、CMMLU、C-Eval、GSM8K、ARC、HellaSwag、TruthfulQA、MATH和HumanEval等；支持多种类型的模型评测，包括LLM、多模态LLM、embedding模型和reranker模型。EvalScope还适用于多种评测场景，如端到端RAG评测、竞技场模式和模型推理性能压测等。此外，通过ms-swift训练框架的无缝集成，可一键发起评测，实现了模型训练到评测的全链路支持。

使用指南：[EvalScope使用指南](https://evalscope.readthedocs.io/zh-cn/latest/user_guides/backend/vlmevalkit_backend.html)

### 环境准备

In [ ]:
!pip install evalscope[vlmeval] -U
!pip install ms-swift -U

### 部署模型

In [ ]:
!CUDA_VISIBLE_DEVICES=0 swift deploy --model_type internvl2-8b --port 8000

In [3]:
task_cfg_dict = {
    'eval_backend': 'VLMEvalKit',
    'eval_config': {
        'data': ['SEEDBench_IMG', 'ChartQA_TEST'],
        'limit': 30,
        'mode': 'all',
        'model': [{
            'api_base': 'http://localhost:8000/v1/chat/completions',
            'key': 'EMPTY',
            'name': 'CustomAPIModel',
            'temperature': 0.0,
            'type': 'internvl2-8b'
        }],
        'reuse': False,
        'work_dir': 'outputs',
        'judge': 'exact_matching'
    }
}

from evalscope.run import run_task
from evalscope.summarizer import Summarizer


def run_eval():
    # 选项 1: python 字典
    task_cfg = task_cfg_dict

    # 选项 2: yaml 配置文件
    # task_cfg = 'eval_openai_api.yaml'

    run_task(task_cfg=task_cfg)

    print('>> Start to get the report with summarizer ...')
    report_list = Summarizer.get_report_from_cfg(task_cfg)
    print(f'\n>> The report list: {report_list}')


run_eval()

2024-11-25 19:48:03,083 - evalscope - INFO - ** Args: Task config is provided with dictionary type. **
2024-11-25 19:48:03,084 - evalscope - INFO - Check VLM Evaluation Kit: Installed
2024-11-25 19:48:03,085 - evalscope - INFO - *** Run task with config: Arguments(data=['SEEDBench_IMG', 'ChartQA_TEST'], model=['internvl2-8b'], fps=-1, nframe=8, pack=False, use_subtitle=False, work_dir='outputs', mode='all', nproc=16, retry=None, judge='exact_matching', verbose=False, ignore=False, reuse=False, limit=30, config=None, OPENAI_API_KEY='EMPTY', OPENAI_API_BASE=None, LOCAL_LLM=None) 

[2024-11-25 19:48:03,085] WARNING - RUN - run.py: run_task - 145: --reuse is not set, will not reuse previous (before one day) temporary files
2024-11-25 19:48:03,085 - RUN - WARNING - --reuse is not set, will not reuse previous (before one day) temporary files
[2024-11-25 19:48:07,410] INFO - ChatAPI - gpt.py: __init__ - 135: Using API Base: http://localhost:8000/v1/chat/completions; API Key: EMPTY
2024-11-25 

>> Start to get the report with summarizer ...

>> The report list: [{'internvl2-8b_SEEDBench_IMG_acc': {'split': 'none', 'Overall': '0.6333333333333333', 'Instance Attributes': '0.8571428571428571', 'Instance Identity': '0.3333333333333333', 'Instance Interaction': '1.0', 'Instance Location': '0.0', 'Instances Counting': '0.5', 'Scene Understanding': '0.75', 'Visual Reasoning': '1.0'}}, {'internvl2-8b_ChartQA_TEST_acc': {'test_human': '53.333333333333336', 'Overall': '53.333333333333336'}}]


## 2. 直接使用VLMEvalKit

直接使用VLMEvalKit需设置`VLMEVALKIT_USE_MODELSCOPE=1`来开启从modelscope下载数据集的能力，目前支持如下视频评测数据集：
- MVBench_MP4
- MLVU_OpenEnded
- MLVU_MCQ
- LongVideoBench
- TempCompass_MCQ
- TempCompass_Captioning
- TempCompass_YorN
- Video-MME
- MVBench
- MMBench-Video

### 环境准备

In [ ]:

git clone https://github.com/open-compass/VLMEvalKit.git
cd VLMEvalKit
pip install -e .

VLM 配置：所有 VLMs 都在 `vlmeval/config.py` 中配置。对于某些 VLMs（如 MiniGPT-4、LLaVA-v1-7B），需要额外的配置（在配置文件中配置代码 / 模型权重根目录）。在评估时，你应该使用 `vlmeval/config.py` 中 supported_VLM 指定的模型名称来选择 VLM。确保在开始评估之前，你可以成功使用 VLM 进行推理，使用以下命令 `vlmutil check {MODEL_NAME}`。

In [ ]:
# 执行如下bash命令开始评测：
!python run.py --data TempCompass --model InternVL2-8B